In [2]:
import numpy as np
import glob
import keras
from keras.utils.np_utils import to_categorical
from keras.layers import Dense,Dropout,Activation,LSTM
from keras.models import Model,Sequential
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [3]:
from music21 import stream
from music21 import converter, instrument,note,chord

In [4]:
#Reading Files from midi songs folder -----------------Here we extract notes from the audio files and 
#append all the notes list notes[]
notes=[]
i=1
for file in glob.glob("C:/Users/user/cpc/midi_songs/*.mid"):
    print("Reading File No.:", i)
    i+=1
    midi= converter.parse(file)        #Parses the file audio
    #print(midi)
    notes_to_parse=[]
    parts=instrument.partitionByInstrument(midi)  # Extract the piano tunes
    #print(parts)
    if parts:
        notes_to_parse=parts.parts[0].recurse()
    else:                                             #if there are piano tunes we recurse else include flat notes
        notes_to_parse= midi.flat_notes
        
    #print(notes_to_parse)
    for element in notes_to_parse:
        if isinstance(element, note.Note):               #For every tune of notes we extract notes and chords
            notes.append(str(element.pitch))
        elif isinstance( element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))


Reading File No.: 1
Reading File No.: 2
Reading File No.: 3
Reading File No.: 4
Reading File No.: 5
Reading File No.: 6
Reading File No.: 7
Reading File No.: 8
Reading File No.: 9
Reading File No.: 10
Reading File No.: 11
Reading File No.: 12
Reading File No.: 13
Reading File No.: 14
Reading File No.: 15
Reading File No.: 16
Reading File No.: 17
Reading File No.: 18
Reading File No.: 19
Reading File No.: 20
Reading File No.: 21
Reading File No.: 22
Reading File No.: 23
Reading File No.: 24
Reading File No.: 25
Reading File No.: 26
Reading File No.: 27
Reading File No.: 28
Reading File No.: 29
Reading File No.: 30
Reading File No.: 31
Reading File No.: 32
Reading File No.: 33
Reading File No.: 34
Reading File No.: 35
Reading File No.: 36
Reading File No.: 37
Reading File No.: 38
Reading File No.: 39
Reading File No.: 40
Reading File No.: 41
Reading File No.: 42
Reading File No.: 43
Reading File No.: 44
Reading File No.: 45
Reading File No.: 46
Reading File No.: 47
Reading File No.: 48
R

In [11]:
print(notes[:4])

['4.9', 'E2', '4.9', '4.9']


In [12]:
seq_len=100
pitchnames=sorted(set(item for item in notes))          #Extracting sorted and unique notes
note2int = dict((note,number) for number,note in enumerate(pitchnames))      #Vocab for model

In [13]:
net_i=[]
net_o=[]
# Here we create our examples for training which consisits of 100 length of notes and the next note as its target
# We also tokenize our input examples using vocab note2int
for i in range(0,len(notes)-seq_len,1):
    seqi=notes[i:i+seq_len]
    #print(seqi)
    seqo=notes[i+seq_len]
    #print(seqi)
    #print(seqo)
    
    net_i.append([note2int[t] for t in seqi])
    
    net_o.append([note2int[seqo]])

In [14]:

n_patterns=len(net_i)
n_vocab=len(note2int)
net_i= np.reshape(net_i,(n_patterns,seq_len,1))
net_i=net_i/float(n_vocab)


In [15]:
net_o=to_categorical(net_o)
net_o

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [16]:
print(net_o.shape)
print(net_i.shape)



(57077, 358)

In [21]:
#Creating our Deep Learning Model

def make_model():
    model = Sequential()
    model.add(LSTM(512,input_shape=(net_i.shape[1], net_i.shape[2]),return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=False))
    model.add(Dense(256))#activation='relu')
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))#,activation='softmax')
    model.add(Activation('softmax'))
    return model
model = make_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 100, 512)          1052672   
_________________________________________________________________
dropout_3 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 100, 512)          2099200   
_________________________________________________________________
dropout_4 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
activation_3 (Activation)    (None, 256)               0         
__________

In [22]:
# Checkpoint created Keeps track of weights after every epoch 
filepath = "weights-improvement-={epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
callbacks_list = [checkpoint]


In [23]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [24]:
#The Model was trained for 10 hrs on a GPU on kaggle kernel for 80 epochs after which I managed to get an accuracy of approx
#  88% which is not bad i guess

model.load_weights('../pianofiles/3lstm_65_acc_88.h5')

In [1]:
#model.fit(net_i, net_o, epochs=1, batch_size=64,callbacks=callbacks_list)

In [ ]:
model.save_weights("weights_epoch80_acc_82.h5")

In [25]:
#Now that we have trained our model we now try to predict the tunes

start= np.random.randint(0,len(net_i)-1) # Take a random sample from input
pattern= net_i[start]


In [27]:
int2not2=dict((note2int[x],x) for x in note2int) #Reverse vocab

In [28]:
pred_out=[]

In [29]:
#WE predict the next 100 note based on the given pattern 

for ni in range(100):
    predi=np.reshape(pattern,(1,len(pattern),1))  #Reshape the given pattern
    prediction= model.predict(predi)               #Obtain the predicted outputs
    i=np.argmax(prediction)                            #Get the best note with maximum value
    res=int2not2[i]                              
    pred_out.append(res)                           #Append the note to output notes
    
    pattern=np.append(pattern,i/len(note2int)) 
    pattern=pattern[1:]                            #Since pattern has increased by 1 note we ignore the 1st note

In [31]:
print(len(pred_out))

200


In [ ]:
pred_out[:10]

In [ ]:
#Now that we have created the output notes from model we need to convert it into
# actual notes for midi file as they are currently in string
offset = 0
output_notes = []

for pattern in pred_out:
    #If its a chord like 3.4 or 6.4.2
    if(('.' in pattern) or pattern.isdigit()):
        
        
        notesinchord=pattern.split(".")
        notes=[]
        for cur in notesinchord:                              #set parameters of the chord
            newnote=note.Note(int(cur))
            newnote.storedInstrument=instrument.Piano()
            notes.append(newnote)
        newchord= chord.Chord(notes)
        newchord.offset= offset
        output_notes.append(newchord)
    else:                                                  #else if pattern is a note simply append it as note
        newnote=note.Note(pattern)
        newnote.offset = offset
        newnote.storedInstrument = instrument.Piano()
        output_notes.append(newnote)
    
    offset+=1.0                                     #Offset keeps track of time between two note play
                                                    #Like offset 1.0 here tells that after each noteplay theres a gap of 1s
    
        

    

In [49]:
#Congrats you're all set you just predicted a tone from a input example 
#Now only thing remaining is to convert in into a midi file for listening it
#which can be easily done by streams

midistream= stream.Stream(output_notes)
midistream.write('midi',fp='C:/Users/user/predction.mid')

'C:/Users/user/predction.mid'